# FHE prediction over MLToolbox generated model
Expected RAM usage: 40 GB.
Expected runtime: 50 minutes.

This notebook should be run after running `19_MLToolbox.ipynb` notebook. It loads the model and input samples generated by `19_MLToolbox.ipynb`, and runs prediction on the plain MLToolbox model as well as on an FHE encryption of the MLToolbox model. Finally, the plaintext and FHE prediction outputs are compared to make sure they are similar.

## Step 1. Prediction on plaintext model

### 1.1 Imports and initializations

In [ ]:
from datetime import datetime
import json
import numpy as np
import os
import pyhelayers
import torch

When `debug_mode` flag is on, the FHE prediction is done using a `Mockup` encryption scheme, which holds everything in plaintext but applies the same operations that are applied in case of a real encryption. Setting this flag to `True` will reduce the runtime of the notebook, which is useful for testing purposes.

In [ ]:
debug_mode = False 

### 1.2 Load input sampels
We assume that `19_MLToolbox.ipynb` notebook has been run prior to running this block.

In [ ]:
num_samples = 1
plain_samples = torch.load('../outputs/mltoolbox/plain_samples.pt')
plain_samples = plain_samples[:num_samples]
batch_size = len(plain_samples)

### 1.3 Run prediction on MLToolbox plaintext model
We Load the MLToolbox model generated by `19_MLToolbox.ipynb` notebook and run prediction with this model.

In [ ]:
checkpoint = torch.load(os.path.join('../outputs/mltoolbox/polynomial/resnet18_postproc_checkpoint.pth.tar'), map_location=torch.device('cpu'))
model = checkpoint['model']
plain_model_predictions = model(plain_samples).detach().numpy()
plain_predicted_labels = np.argmax(plain_model_predictions, 1)

## Step 2. FHE prediction

### 2.1 Load NN architecture and weights using the FHE library

We use the `init_from_onnx_file` function of the `NeuralNetPlain` class to load the NN model.

In [ ]:
model_path = '../outputs/mltoolbox/polynomial/resnet18.onnx'
nnp = pyhelayers.NeuralNetPlain()
hyper_params = pyhelayers.PlainModelHyperParams()
nnp.init_from_files(hyper_params,[model_path])

### 2.2 Compile

Using HE can require configuring complex and non-intuitive parameters. Luckily, helayers has an `Optimizer` tool offers an automatic optimization process that analyzes the model, and tunes various HE parameters to work best for the given scenario.

The optimizer runs when we run 'compile' on the plain model. It receives also run requirements, some simple and intuitive input from the user (e.g., the desired security level). As output, it produces a `profile` object which contains all the details related to the HE configuration and packing. These details are automatically selected to ensure optimal performance given the user's requests.

In [ ]:
he_run_req = pyhelayers.HeRunRequirements()
# Use the HEaaN context as the underlying FHE
he_run_req.set_he_context_options([pyhelayers.HeaanContext()])
# The encryption is at least as strong as 128-bit encryption.
he_run_req.set_security_level(128)
# Our numbers are theoretically stored with a precision of about 2^-40.
he_run_req.set_fractional_part_precision(30)
# The model weights are kept in the plain
he_run_req.set_model_encrypted(False)
# Activate lazy encoding of the weights to save memory
he_run_req.set_lazy_encoding(True)

if debug_mode:
    # In debug mode, we use a fixed tile layout to reduce compilation time
    tile_layout = pyhelayers.TTShape([16, 8, 8, 16, 1])
    he_run_req.set_fixed_tile_layout(tile_layout)
else:
    # The batch size for NN.
    he_run_req.optimize_for_batch_size(batch_size)

# Compile - run the optimizer
profile = pyhelayers.HeModel.compile(nnp, he_run_req)

profile_as_json = profile.to_string()
# Profile supports I/O operations and can be stored on file.
print(json.dumps(json.loads(profile_as_json), indent=4))

### 2.3 Initialize the context, and encrypt the NN
Now we initialize the context object and encrypt the neural network using our profile object.

In [ ]:
if debug_mode:
    profile.set_not_secure_mockup()
    
context=pyhelayers.HeModel.create_context(profile)
nn = pyhelayers.NeuralNet(context)
nn.encode(nnp, profile)

### 2.4 Encrypt the input samples
Here, we encrypt the samples we're going to be running an inference on. The data is encrypted by the iop object (input output processor), which contains model meta data only, and can process inputs and outputs of the model.

In [ ]:
iop=nn.create_io_processor()
encrypted_samples = pyhelayers.EncryptedData(context)
iop.encode_encrypt_inputs_for_predict(encrypted_samples, [plain_samples])

### 2.5 Run prediction over encrypted data, using the encrypted model

In [ ]:
encrypted_predictions = pyhelayers.EncryptedData(context)
predict_start = datetime.now()
nn.predict(encrypted_predictions, encrypted_samples)
predict_end = datetime.now()
print('prediction time = %d seconds', predict_end - predict_start)

### 2.6 Decrypt the prediction result
The final labels are computed as the argmax of the predicted probabilities.

In [ ]:
fhe_model_predictions = iop.decrypt_decode_output(encrypted_predictions)
fhe_predicted_labels = np.argmax(fhe_model_predictions, 1)

### 2.7 Compare the predictions of the encrypted model with the predictions of the plain model
The FHE model's predictions are shown to match those produced by the plain model.

In [ ]:
print('labels predicted by the FHE model: ', fhe_predicted_labels)
print('labels predicted by the plain model: ', plain_predicted_labels)
np.testing.assert_array_equal(fhe_predicted_labels, plain_predicted_labels)